In [1]:
import os

import numpy as np
import pandas as pd

import cloudpickle

from transformers import *
import torch

2021-11-08 21:25:16.432292: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2021-11-08 21:25:16.432402: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/opt/conda/lib/python3.7/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


In [2]:
matchings = pd.read_csv('../input/combiner-target-dataset-0-of-5/final_matchings.csv', keep_default_na=False)

matchings.head(3)

,image_url,page_title,caption_reference_description,count,spaced_filename,spaced_filename_translit,page_title_translit,caption_translit,page_title_lang,page_title_lang_p,...,page_title_contains_digit,undigit_page_title,target,target_id,caption_translation,final_caption,PREfinal_caption,page_title_translation,final_page_title,PREfinal_page_title
0,https://upload.wikimedia.org/wikipedia/commons...,Jürgen Ovens,"Jürgen Ovens's Justitia, 1663-1665, Museumsber...",4,Jurgen ovens justitia,,Juergen Ovens,"Juergen Ovens's Justitia, 1663-1665, Museumsbe...",en,0.124504,...,False,Jürgen Ovens,"Jürgen Ovens [SEP] Jürgen Ovens's Justitia, 16...",0,,"Jürgen Ovens's Justitia, 1663-1665, Museumsber...","Jürgen Ovens's Justitia, 1663-1665, Museumsber...",,Jürgen Ovens,Jürgen Ovens
1,https://upload.wikimedia.org/wikipedia/commons...,Jürgen Ovens,"Jürgen Ovens: Justitia, 1663-1665, Museumsberg...",4,Jurgen ovens justitia,,Juergen Ovens,"Juergen Ovens: Justitia, 1663-1665, Museumsber...",en,0.124504,...,False,Jürgen Ovens,"Jürgen Ovens [SEP] Jürgen Ovens: Justitia, \n ...",1,,"Jürgen Ovens: Justitia, 1663-1665, Museumsberg...","Jürgen Ovens: Justitia, 1663-1665, Museumsberg...",,Jürgen Ovens,Jürgen Ovens
2,https://upload.wikimedia.org/wikipedia/commons...,"Овенс, Юрген","Юрген Овенс Юстиция, 1663—1665, Музей Фленсбур...",4,Jurgen ovens justitia,,"Ovens, Jurgen","Jurgen Ovens Justitsija, 1663—1665, Muzej Flen...",ru,0.647271,...,False,"Овенс, Юрген","Овенс, Юрген [SEP] Юрген Овенс Юстиция, 1663—1...",2,"Jurgen Owens Justice, -, Flensburg Museum. Owe...","Jurgen Owens Justice, -, Flensburg Museum. Owe...","Jurgen Owens Justice, -, Flensburg Museum. Owe...","Owens, Jürgen","Owens, Jürgen","Owens, Jürgen"


In [3]:
matchings = matchings[matchings['caption_lang'].isin(['en', 'de', 'it', 'fr'])]

In [4]:
model = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

label_list = [
    "O",       # Outside of a named entity
    "B-MISC",  # Beginning of a miscellaneous entity right after another miscellaneous entity
    "I-MISC",  # Miscellaneous entity
    "B-PER",   # Beginning of a person's name right after another person's name
    "I-PER",   # Person's name
    "B-ORG",   # Beginning of an organisation right after another organisation
    "I-ORG",   # Organisation
    "B-LOC",   # Beginning of a location right after another location
    "I-LOC"    # Location
]

Downloading:   0%|          | 0.00/998 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [5]:
def retrieve_entities(pairs, verbose=False):
    def repair(s):
        return s.replace(' #', '').replace('#', '')
    def unique(l):
        return list(set(l))
    
    
    reading_entity = False
    current_entity_type = None
    
    misc, per, org, loc = [], [], [], []
    
    entity_parts = []
    
    for part, et in pairs:
        if not reading_entity and et == 'O':
            continue
        else:
            if not reading_entity:
                current_et = et
                reading_entity = True               
                
            if current_et == et:
                entity_parts.append(part)
            else:
                repaired_entity = repair(' '.join(entity_parts))
                
                if verbose:
                    print(repaired_entity)
                    
                locals()[current_et.split('-')[-1].lower()].append(repaired_entity)
                entity_parts.clear()
                
                if et == 'O':
                    reading_entity = False
                    current_et = None
                else:
                    entity_parts.append(part)
                    current_et = 'I-' + et.split('-')[-1]
                    
                    
    return { 'misc': unique(misc), 'per': unique(per), 'org': unique(org), 'loc': unique(loc)}

In [6]:
image_entities = { }

for index, row in matchings.iterrows():
    try:
        sequence = row['caption_reference_description'].strip()
        
        if sequence != '':
            tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(sequence)))
            inputs = tokenizer.encode(sequence, return_tensors="pt")

            outputs = model(inputs)[0]
            predictions = torch.argmax(outputs, dim=2)

            l = [(token, label_list[prediction]) for token, prediction in zip(tokens, predictions[0].tolist())]

            image_entities[index] = retrieve_entities(l)
    except Exception as ex:
        print(index, sequence)
        print(str(ex))
        print()

Token indices sequence length is longer than the specified maximum sequence length for this model (662 > 512). Running this sequence through the model will result in indexing errors


270165 Decriminalization of same-sex sexual activity by country or territory 1790–1799 1800–1829 1830–1839 1840–1859 1860–1869 1870–1879 1880–1889 1890–1929 1930–1939 1940–19491 1950–1959 1960–1969 1970–1979 1980–1989 1990–19992 2000–2009 2010–present Same-sex sexual activity legal3 Male same-sex sexual activity illegal Same-sex sexual activity illegal Notes 1: During World War II, Nazi Germany annexed territory or established reichskommissariats which extended Germany's laws against same-sex sexual activity to those territories and reichskommissariats. Same-sex sexual activity was previously legalized in the following countries or territories before German annexation or establishment of reichskommissariats: Bas-Rhin (legal in 1791), Belgium (legal in 1795), Belluno (legal in 1890), Friuli-Venezia Giulia (legal in 1890), Haut-Rhin (legal in 1791), Luxembourg (legal in 1795), Moselle (legal in 1791), Netherlands (legal in 1811), Nord (legal in 1791), Pas-de-Calais (legal in 1791), Polan

In [7]:
with open('caption_lang_entities.dump', 'wb') as file:
    cloudpickle.dump(image_entities, file)